# 태양 고도 정보 불러오기
* [태양 고도 API 주소](https://data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15012692)
* 방위각, 고도를 불러올 예정

In [7]:
import pandas as pd
from urllib.parse import urlencode, quote_plus
from urllib.request import urlopen
import json
import xmltodict
import re

In [2]:
key = input('your api key >> ')

In [10]:
# url에서 응답 불러오기

def get_sun_json(region, date, key):
    '''
    region: ulsan or dangjin
    date: 년월일, ex-'20210606'
    key: your api key
    '''

    lat_long = {'ulsan': (35.4776509, 129.380778), 'dangjin': (37.0507527, 126.5102993)}
    
    url_base = 'http://apis.data.go.kr/B090041/openapi/service/SrAltudeInfoService/getLCSrAltudeInfo'
    queryParams = urlencode({
        quote_plus('locdate'): date,                    # 날짜
        quote_plus('latitude'): lat_long[region][0],    # 위도
        quote_plus('longitude'): lat_long[region][1],   # 경도
        quote_plus('dnYn'): 'y',                        # 위도경도 숫자 형식 체크
    })

    url = f'{url_base}?serviceKey={key}&{queryParams}'
    response = urlopen(url)
    results = response.read().decode('utf-8')
    
    results_json = xmltodict.parse(results)
    data = json.loads(json.dumps(results_json))
    
    return data


def angle_change(angle_str):
    
    angle_num = re.findall('\d+', angle_str)
    
    angle = int(angle_num[0]) + (1/60) * int(angle_num[1])
    
    return angle


def get_sun_df(region, date, key):
    
    data_json = get_sun_json(region, date, key)
    data = data_json['response']['body']['items']['item']
    
    df = pd.DataFrame(columns=['date', 'altitude', 'azimuth', 'altitude_s'])
    
    # 데이터 프레임 시간마다 추가
    hours = ['09', '12', '15', '18']
    for hour in hours:
        date = data['locdate'] + f' {hour}:00:00'
        altitude = angle_change(data[f'altitude_{hour}'])
        azimuth = angle_change(data[f'azimuth_{hour}'])
        altitude_s = angle_change(data['altitudeMeridian'])
        
        df_tmp = pd.DataFrame({
            'date': [date],
            'altitude': [altitude],
            'azimuth': [azimuth],
            'altitude_s': [altitude_s],
        })
        df = pd.concat([df, df_tmp])
    
    return df
    

In [4]:
date_list = []
for a in pd.date_range('2018-03-01', '2021-02-28'):
    date_list.append(f'{a.year}{a.month:0>2}{a.day:0>2}')

date_list

['20180301',
 '20180302',
 '20180303',
 '20180304',
 '20180305',
 '20180306',
 '20180307',
 '20180308',
 '20180309',
 '20180310',
 '20180311',
 '20180312',
 '20180313',
 '20180314',
 '20180315',
 '20180316',
 '20180317',
 '20180318',
 '20180319',
 '20180320',
 '20180321',
 '20180322',
 '20180323',
 '20180324',
 '20180325',
 '20180326',
 '20180327',
 '20180328',
 '20180329',
 '20180330',
 '20180331',
 '20180401',
 '20180402',
 '20180403',
 '20180404',
 '20180405',
 '20180406',
 '20180407',
 '20180408',
 '20180409',
 '20180410',
 '20180411',
 '20180412',
 '20180413',
 '20180414',
 '20180415',
 '20180416',
 '20180417',
 '20180418',
 '20180419',
 '20180420',
 '20180421',
 '20180422',
 '20180423',
 '20180424',
 '20180425',
 '20180426',
 '20180427',
 '20180428',
 '20180429',
 '20180430',
 '20180501',
 '20180502',
 '20180503',
 '20180504',
 '20180505',
 '20180506',
 '20180507',
 '20180508',
 '20180509',
 '20180510',
 '20180511',
 '20180512',
 '20180513',
 '20180514',
 '20180515',
 '20180516',

In [11]:
df_ulsan = pd.DataFrame()

for date in date_list:
    df1 = get_sun_df('ulsan', date, key)
    df_ulsan = pd.concat([df_ulsan, df1])

df_ulsan
    

,date,altitude,azimuth,altitude_s
0,20180301 09:00:00,23.500000,119.000000,47.183333
0,20180301 12:00:00,46.333333,166.983333,47.183333
0,20180301 15:00:00,35.416667,225.600000,47.183333
0,20180301 18:00:00,2.933333,258.633333,47.183333
0,20180302 09:00:00,23.800000,118.750000,47.566667
...,...,...,...,...
0,20210227 18:00:00,2.616667,258.050000,46.550000
0,20210228 09:00:00,23.283333,119.200000,46.916667
0,20210228 12:00:00,46.066667,167.000000,46.916667
0,20210228 15:00:00,35.216667,225.383333,46.916667


In [6]:
df_dangjin = pd.DataFrame()

for date in date_list:
    df2 = get_sun_df('dangjin', date, key)
    df_dangjin = pd.concat([df_dangjin, df2])
    
df_dangjin

,date,altitude,azimuth
0,20180301 09:00:00,20.866667,118.283333
0,20180301 12:00:00,43.616667,164.700000
0,20180301 15:00:00,34.866667,222.350000
0,20180301 18:00:00,4.083333,257.233333
0,20180302 09:00:00,21.183333,118.033333
...,...,...,...
0,20210227 18:00:00,3.750000,256.666667
0,20210228 09:00:00,20.650000,118.466667
0,20210228 12:00:00,43.333333,164.733333
0,20210228 15:00:00,34.650000,222.133333


In [8]:
df_dangjin

,date,altitude,azimuth
0,20180301 09:00:00,20.866667,118.283333
0,20180301 12:00:00,43.616667,164.700000
0,20180301 15:00:00,34.866667,222.350000
0,20180301 18:00:00,4.083333,257.233333
0,20180302 09:00:00,21.183333,118.033333
...,...,...,...
0,20210227 18:00:00,3.750000,256.666667
0,20210228 09:00:00,20.650000,118.466667
0,20210228 12:00:00,43.333333,164.733333
0,20210228 15:00:00,34.650000,222.133333


# 이하 생략

In [48]:
get_sun_df('ulsan', '20180301', key)

,date,altitude,azimuth
0,20180301 09:00:00,23.500000,119.000000
0,20180301 12:00:00,46.333333,166.983333
0,20180301 15:00:00,35.416667,225.600000
0,20180301 18:00:00,2.933333,258.633333


In [34]:
item

{'altitudeMeridian': '47˚ 11´',
 'altitude_09': '23˚ 30´',
 'altitude_12': '46˚ 20´',
 'altitude_15': '35˚ 25´',
 'altitude_18': '2˚ 56´',
 'azimuth_09': '119˚ 0´',
 'azimuth_12': '166˚ 59´',
 'azimuth_15': '225˚ 36´',
 'azimuth_18': '258˚ 38´',
 'latitude': '3511',
 'latitudeNum': '35.1833330',
 'location': '부산',
 'locdate': '20180301',
 'longitude': '12904',
 'longitudeNum': '129.0666660'}

['20180301',
 '20180302',
 '20180303',
 '20180304',
 '20180305',
 '20180306',
 '20180307',
 '20180308',
 '20180309',
 '20180310',
 '20180311',
 '20180312',
 '20180313',
 '20180314',
 '20180315',
 '20180316',
 '20180317',
 '20180318',
 '20180319',
 '20180320',
 '20180321',
 '20180322',
 '20180323',
 '20180324',
 '20180325',
 '20180326',
 '20180327',
 '20180328',
 '20180329',
 '20180330',
 '20180331',
 '20180401',
 '20180402',
 '20180403',
 '20180404',
 '20180405',
 '20180406',
 '20180407',
 '20180408',
 '20180409',
 '20180410',
 '20180411',
 '20180412',
 '20180413',
 '20180414',
 '20180415',
 '20180416',
 '20180417',
 '20180418',
 '20180419',
 '20180420',
 '20180421',
 '20180422',
 '20180423',
 '20180424',
 '20180425',
 '20180426',
 '20180427',
 '20180428',
 '20180429',
 '20180430',
 '20180501',
 '20180502',
 '20180503',
 '20180504',
 '20180505',
 '20180506',
 '20180507',
 '20180508',
 '20180509',
 '20180510',
 '20180511',
 '20180512',
 '20180513',
 '20180514',
 '20180515',
 '20180516',